In [2]:
import torch 
import numpy as np 
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
        self.softmax = nn.Softmax(dim=1)                # NEW ADDITIION

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        x = self.softmax(x)                             # NEW ADDITION
        return x

In [24]:
model = AlexNet().to("cuda:3")

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)

In [6]:
pwd

'/app/timeseries/imagenet'

In [8]:
checkpoint = torch.load('/app/timeseries/imagenet/trained-models/af1_checkpoint.pth.tar')

In [9]:
print(checkpoint.keys())

dict_keys(['epoch', 'state_dict', 'best_acc1', 'optimizer'])


In [10]:
print(checkpoint['best_acc1'])
# print(checkpoint['optimizer'])
print(checkpoint['epoch'])

tensor(0.1000, device='cuda:0')
90


In [11]:
# # create new OrderedDict that does not contain `module.`
# from collections import OrderedDict
# new_state_dict = OrderedDict()
# for k, v in checkpoint['state_dict'].items():
#     name = k[7:] # remove `module.`
#     new_state_dict[name] = v
# # load params
# model.load_state_dict(new_state_dict)
# model.eval()

In [12]:
model.load_state_dict(checkpoint['state_dict'])
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [13]:
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [14]:
valdir = os.path.join('/app/timeseries/imagenet/data', 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

In [15]:
val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(valdir, transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

In [17]:
from grace_torchconfusion import mean_f1_approx_loss_on

In [23]:
criterion = mean_f1_approx_loss_on(
        device="cuda:3", thresholds=torch.arange(0.1, 1, 0.1).cuda())

In [33]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

def validate(val_loader, model, criterion):
    # switch to evaluate mode
    model.eval()
    acc1_arr, acc5_arr = [], [] 
    with torch.no_grad():
        for i, (images, target) in enumerate(val_loader):
            print("Calculating batch...{}".format(i))
            images = images.to(device)
            target = target.to(device)

            # compute output
            output = model(images)
            _, predicted = torch.max(output, 1)
            print(target)
            
            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            acc1_arr.append(acc1.detach().item())
            acc5_arr.append(acc5.detach().item())
            
    return acc1_arr, acc5_arr
            

In [34]:
acc1_arr, acc5_arr = validate(val_loader, model, criterion)

Exception ignored in: Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f8214152630>>
<bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f8214152630>>Exception ignored in: Traceback (most recent call last):
Exception ignored in: 
<bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f8214152630>><bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f8214152630>>  File "/app/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
Traceback (most recent call last):


  File "/app/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
Traceback (most recent call last):
    Traceback (most recent ca

Calculating batch...0
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:3')
Calculating batch...1
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 5, 5, 5, 5, 5, 5], device='cuda:3')
Calculating batch...2
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5,

Calculating batch...16
tensor([40, 40, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41,
        41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41,
        41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 42, 42,
        42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42,
        42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42,
        42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 43, 43, 43, 43, 43, 43,
        43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43,
        43, 43], device='cuda:3')
Calculating batch...17
tensor([43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43,
        43, 43, 43, 43, 43, 43, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44,
        44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44,
        44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44,
        44, 44, 45, 45, 45, 45, 45, 45, 

Calculating batch...30
tensor([76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 77, 77, 77, 77, 77, 77, 77, 77,
        77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77,
        77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77,
        77, 77, 77, 77, 77, 77, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
        78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
        78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
        78, 78, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79,
        79, 79], device='cuda:3')
Calculating batch...31
tensor([79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79,
        79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 80, 80, 80, 80,
        80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 80, 80, 80, 80, 80, 80, 

KeyboardInterrupt: 

In [36]:
acc5_arr

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 15.625,
 23.4375,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 18.75,
 20.3125,
 23.4375,
 15.625,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 39.0625,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [34]:
np.mean(np.array(acc1_arr))

0.09990409207161126

In [37]:
np.mean(np.array(acc5_arr))

0.49952046035805625

In [36]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

def validate_v2(val_loader, model, criterion):
    # switch to evaluate mode
    model.eval()
    outputs, targets = [], []
    with torch.no_grad():
        for i, (images, target) in enumerate(val_loader):
            print("Calculating batch...{}".format(i))
            images = images.to(device)
            target = target.to(device)

            # compute output
            output = model(images)
            
            outputs.append(output.cpu().numpy())
            targets.append(target.cpu().numpy())
    return outputs, targets

In [37]:
outputs, targets = validate_v2(val_loader, model, criterion)

Calculating batch...0
Calculating batch...1
Calculating batch...2
Calculating batch...3
Calculating batch...4
Calculating batch...5
Calculating batch...6
Calculating batch...7
Calculating batch...8
Calculating batch...9
Calculating batch...10
Calculating batch...11
Calculating batch...12
Calculating batch...13
Calculating batch...14
Calculating batch...15
Calculating batch...16
Calculating batch...17
Calculating batch...18
Calculating batch...19
Calculating batch...20
Calculating batch...21
Calculating batch...22
Calculating batch...23
Calculating batch...24
Calculating batch...25
Calculating batch...26
Calculating batch...27
Calculating batch...28
Calculating batch...29
Calculating batch...30
Calculating batch...31
Calculating batch...32
Calculating batch...33
Calculating batch...34
Calculating batch...35
Calculating batch...36
Calculating batch...37
Calculating batch...38
Calculating batch...39
Calculating batch...40
Calculating batch...41
Calculating batch...42
Calculating batch...4

Calculating batch...348
Calculating batch...349
Calculating batch...350
Calculating batch...351
Calculating batch...352
Calculating batch...353
Calculating batch...354
Calculating batch...355
Calculating batch...356
Calculating batch...357
Calculating batch...358
Calculating batch...359
Calculating batch...360
Calculating batch...361
Calculating batch...362
Calculating batch...363
Calculating batch...364
Calculating batch...365
Calculating batch...366
Calculating batch...367
Calculating batch...368
Calculating batch...369
Calculating batch...370
Calculating batch...371
Calculating batch...372
Calculating batch...373
Calculating batch...374
Calculating batch...375
Calculating batch...376
Calculating batch...377
Calculating batch...378
Calculating batch...379
Calculating batch...380
Calculating batch...381
Calculating batch...382
Calculating batch...383
Calculating batch...384
Calculating batch...385
Calculating batch...386
Calculating batch...387
Calculating batch...388
Calculating batc

In [38]:
print

391


In [39]:
flat_outputs = [item for sublist in outputs for item in sublist]
flat_targets = [item for sublist in targets for item in sublist]

In [42]:
flat_outputs = torch.FloatTensor(flat_outputs)
flat_targets = torch.FloatTensor(flat_targets)

In [43]:
acc1, acc5 = accuracy(flat_outputs, flat_targets, topk=(1, 5))

In [52]:
print("Accuracy: {}".format(acc1))
print("Error Rate: {}".format(100-acc1))

Accuracy: tensor([49.9020])
Error Rate: tensor([50.0980])


In [53]:
print("Accuracy: {}".format(acc5))
print("Error Rate: {}".format(100-acc5))

Accuracy: tensor([73.8800])
Error Rate: tensor([26.1200])
